# Temporal Matrix Factorization for Multivariate Time Series Forecasting

Temporal matrix factorization is an important variant in the large family of matrix factorization models. Thanks to the temporal modeling technique (for example vector autoregressive (VAR) process), temporal matrix factorization in extremely useful for multivariate time series forecasting in the presence of missing values. 
The basic idea behind the model is that matrix factorization can learn low-rank temporal patterns from partially observed time series data, while temporal modeling can capture time-evolving coefficients. 

## Matrix Factorization

Is a powerfull tool for reconstructing data matrices with missing entries. The fundamental idea is that a partially observed data matrix can be factorized into two factor matrices of relatively low-rank R. In practice, the common dynamics of large amount of time series could stem from a relativelly small number of latent temporal factors. 

Given a multivariate time series in the form of matrix with N rows and T columns:

![](https://miro.medium.com/max/700/1*UOid48ZhLXl8585baaMtow.png)

In some cases this data matrix is incomplete with some missing entries. We can define the index set of observed entries $\Omega$ and write down a matrix factorization formula with two factor matrices W and X as follows:

$$ P_{\Omega}(Y) = P_{\Omega}(W^{T}X)$$

Where W is of size R by N and X is of the size R by T. Herein, R is the low rank. We can achieve such approximation by constructing the optimization problem with a certain loss function:

$$ min_{W,X} 0.5 * \lVert P_{\Omega}(Y-W^{T}X) \rVert_{F}^2 + \rho (\lVert W \rVert_{F}^2 + \lVert X \rVert_{F}^2) $$

On time series data Y, if there exist some unobserved entries, solving such optimization problem can impute missing values. However, if we want to forecast future data points, we should build temporal correlation upon a certain time series model. There are many options for building temporal correlation on time series, including univariate autoregressive process and multivariate VAR process.

## VAR Process

The coefficient matrix in VAR allows us to capture coevolution patterns amoing multiple variables in time series data. Formally, on time series ${x_{1},x_{2},x_{3},...x_{T}}$ a dth-order VAR process can be modeled as:

$$ x_{t} \approx \sum_{k=1}^{d} A_{k}x_{t-k} $$

Until now, we have discussed the basic formulas of both matrix factorization and VAR. One question is that how to develop an elegant connection between matrix factorization and VAR?

## Temporal Matrix Factorization

### Optimization Problem

Recall that the common dynamics of large amounts of time series could stem from a relatively small number of latent temporal factors. Therefore, it is possible to model a temporal factor matrix X by a VAR process. The optimization problem of temporal matrix factorization can be formulated as follows:

$$ min_{W,X,A_{1},A_{2},...,A_{d}}  0.5 * \lVert P_{\Omega}(Y-W^{T}X) \rVert_{F}^2 \rho /2 (\lVert W \rVert_{F}^2 + \lVert X \rVert_{F}^2) + \lambda / 2 \sum_{d+1}^{T} \lVert x_{t} - \sum_{k=1}^{d} A_{k}x_{t-k} \rVert_{2}^2 $$

At first sight, this optimization problem is rather complicated and difficult to solve. To simplify the problem, we can define a sequence of temporal operators:

$$ \Psi_{k} \triangleq [O_{(T-d)*(d-k)} I_{T-d} O_{(T-d)*k}]  \in R^{T-d}*T , k = 0,1,2,...,d$$

With such notations, the optimization problem can be rewritten as follows:

$$ min_{W,X,A}  0.5 * \lVert P_{\Omega}(Y-W^{T}X) \rVert_{F}^2 \rho /2 (\lVert W \rVert_{F}^2 + \lVert X \rVert_{F}^2) + \lambda / 2 \lVert X\Psi_{0}^{T} - A(I_{d} \otimes X)\Psi^{T} \rVert_{F}^2 $$

![](https://miro.medium.com/max/700/1*CzxBxXvkWM6ES4E52VN96Q.png)

### Solution to the algorithm

We can approximate the least squares solution of both factor matrices W and X by using the conjugate [gradient method](https://medium.com/p/7f16cbae18a3). In terms of the coefficient matrix A, there is a least squares solution. Therefore we can use an **alternating minimization algorithm** to solve the optimization problem. 

In [1]:
import numpy as np

def update_cg(ver, r, q, Aq, rold):
    alpha = rold / np.inner(q,Aq)
    var = var + alpha * q
    r = r - alpha * Aq
    rnew = np.inner(r, r)
    q = r + (rnew/rold)*q
    return var, r, q, rnew

def ell_w(ind, W, X, rho):
    return X @ ((W.T @ X) * ind).T + rho * W

def conj_grad_w(sparse_mat, ind, W, X, rho, maxiter = 5):
    rank, dim1 = W.shape
    w = np.reshape(W, -1, order = 'F')
    r = np.reshape(X @ sparse_mat.T - ell_w(ind, W, X, rho), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim1), order = 'F')
        Aq = np.reshape(ell_w(ind, Q, X, rho), -1, order = 'F')
        w, r, q, rold = update_cg(w, r, q, Aq, rold)
    return np.reshape(w, (rank, dim1), order = 'F')

def ell_x(ind, W, X, A, Psi, d, lambda0, rho):
    rank, dim2 = X.shape
    temp = np.zeros((d * rank, Psi[0].shape[0]))
    for k in range(1, d + 1):
        temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
    temp1 = X @ Psi[0].T - A @ temp
    temp2 = np.zeros((rank, dim2))
    for k in range(d):
        temp2 += A[:, k * rank : (k + 1) * rank].T @ temp1 @ Psi[k + 1]
    return W @ ((W.T @ X) * ind) + rho * X + lambda0 * (temp1 @ Psi[0] - temp2)

def conj_grad_x(sparse_mat, ind, W, X, A, Psi, d, lambda0, rho, maxiter = 5):
    rank, dim2 = X.shape
    x = np.reshape(X, -1, order = 'F')
    r = np.reshape(W @ sparse_mat - ell_x(ind, W, X, A, Psi, d, lambda0, rho), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim2), order = 'F')
        Aq = np.reshape(ell_x(ind, W, Q, A, Psi, d, lambda0, rho), -1, order = 'F')
        x, r, q, rold = update_cg(x, r, q, Aq, rold)
    return np.reshape(x, (rank, dim2), order = 'F')

Here, conj_grad_w is used to approximate the least squares solution to the factor matrix W. Since temporal factor matrix X is associated with both matrix factorization and VAR, approximating the least squares solution to the factor matrix X with conjugate gradient (i.e., conj_grad_x) is much more complicated than than updating W. Note that conjugate gradient is an efficient iterative algorithm, and only few iterations are required for reaching convergence.

Then, we can define the temporal operators associated with VAR as follows. The inputs are temporal dimensionality and the order of VAR, i.e., T and d. In the function, the output Psi is a Python list, and there are d + 1 arrays.

In [2]:
def generate_Psi(T, d):
    Psi = []
    for k in range(0, d + 1):
        if k == 0:
            Psi.append(np.append(np.zeros((T - d, d)), np.eye(T - d), axis = 1))
        else:
            Psi.append(np.append(np.append(np.zeros((T - d, d - k)), np.eye(T - d), axis = 1), 
                                 np.zeros((T - d, k)), axis = 1))
    return Psi

Now, we can define a function tmf for temporal matrix factorization. The inputs include sparse (or dense) time series data matrix, rank R of matrix factorization, order d of VAR, and tradeoff parameters lambda0 and rho. The outputs include an estimated time series matrix, factor matrices {W, X}, and coefficient matrix A.

In [3]:
def tmf(sparse_mat, rank, d, lambda0, rho, maxiter = 50):
    dim1, dim2 = sparse_mat.shape
    ind = sparse_mat != 0
    W = 0.01 * np.random.randn(rank, dim1)
    X = 0.01 * np.random.randn(rank, dim2)
    A = 0.01 * np.random.randn(rank, d * rank)
    Psi = generate_Psi(dim2, d)
    temp = np.zeros((d * rank, dim2 - d))
    for it in range(maxiter):
        W = conj_grad_w(sparse_mat, ind, W, X, rho)
        X = conj_grad_x(sparse_mat, ind, W, X, A, Psi, d, lambda0, rho)
        for k in range(1, d + 1):
            temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
        A = X @ Psi[0].T @ np.linalg.pinv(temp)
        mat_hat = W.T @ X
    return mat_hat, W, X, A

In the temporal matrix factorization algorithm, we build a VAR process and perform forecasting on the temporal factor matrix. Here, we can define a function for VAR forecasting.

In [4]:
def var4cast(X, A, d, delta):
    dim1, dim2 = X.shape
    X_hat = np.append(X, np.zeros((dim1, delta)), axis = 1)
    for t in range(delta):
        X_hat[:, dim2 + t] = A @ X_hat[:, dim2 + t - np.arange(1, d + 1)].T.reshape(dim1 * d)
    return X_hat[:, - delta :]

# Forecasting Multivariate Time Series with NonStationarity Temporal Matrix Factorization

[Uber movement](https://movement.uber.com/?lang=es-PE) provides data and tools for cities to more deeply understand and address urban transportation challenges. Uber movement speed data measures hourley street speeds across a city to enable data-driven city planning and decision making. These data are indeed multivariate time series, with N road segments and T time steps (hours) and are featured as high-dimensional, sparse, and non-stationary. To overcome the challenge of predicting, we propose a nonstationary temporal matrix factorization (NoTMF) framework for multivariate time series forecasting on high dimensional and sparse Uber movement speed data. 

We consider to develop a low-rank matrix factorization framework with the following claims:

1. High-dimensional time series could stem from a relatively small number of latent temporal factors.
2. CFor the fact that modern time series usually show nonstationarity (e.g., trend, seasonality, see the following two graphics). It is necessary to consider the nonstationarity issue when constructing a low-rank matrix factorization model.

![](https://miro.medium.com/max/700/1*cpPr22FeFdyO9jflgXa5QA.png)

## Non Stationarity Temporal Matrix Factorization

To generalize the idea of temporal matrix factorization with non stationary time series, we can impose **differencing operations** (first order differencing, seasonal differencing) on temporal factors. 

$$ min_{W,X,[A_{k}]_{k=1}^{d}}  0.5 * \lVert P_{\Omega}(Y-W^{T}X) \rVert_{F}^2 \rho /2 (\lVert W \rVert_{F}^2 + \lVert X \rVert_{F}^2) + \lambda / 2 \sum_{t=d+m+1}^{T} \lVert (x_{t}-x_{t-m}) - \sum_{k=1}^{d} A_{k}(x_{t-k}-x_{t-m-k}) \rVert_{2}^2 $$


To solve the involved optimization problem, we may need to rewrite vector autoregressive process in the form of matrix. Here, we define some temporal operators associated with temporal dimensionality T, order d, and season m. These temporal operators can help reformulate the vector-form VAR as a matrix-form one.

$$ \Psi_{k} \triangleq [O_{(T-d-m)*(d-k)} I_{T-d-m} O_{(T-d-m)*(k+m)}] + [O_{(T-d-m)*(d+m-k)} I_{T-d-m} O_{(T-d-m)*k}]  \in R^{T-d-m}*T , k = 0,1,2,...,d$$

Then: 

$$ \sum_{t=d+m+1}^{T} \lVert (x_{t}-x_{t-m}) - \sum_{k=1}^{d} A_{k}(x_{t-k}-x_{t-m-k}) \rVert_{2}^2 \equiv \lVert X\Psi_{0}^{T} - \sum_{k=1}^{d} A_{k}X\Psi_{k}^{T} \rVert_{F}^2 \triangleq \lVert X\Psi_{0}^{T} - \sum_{k=1}^{d} A(I_{d} \otimes X)\Psi^{T} \rVert_{F}^2 $$

## Solution to NoTMF

We can apply the alternating minimization method to solve the optimization problem in NoTMF:

$$ \frac{\partial f}{\partial W} = -X \Rho_{\Omega}^{T} (Y-W^{T}X) + \rho W = 0 $$

$$ \frac{\partial f}{\partial X} = -W \Rho_{\Omega} (Y-W^{T}X) + \rho X + \lambda \sum_{k=0}^d A_{k}^{T} (\sum_{h=0}^{d} A_{h} X \Psi_{h}^{T})\Psi_{k} = 0 $$

$$ A = X \Psi_{0}^{T} [(I_{d} \otimes X)\Psi^{T}] $$


For each variable, one thing we need to do is deriving the closed-form solution. With respect to both W and A, it would be easy.But for X there is a complicated matrix equation:


**We can: Solve generalized Sylvester equation (w.r.t X):**

$$ W \Rho_{\Omega}(W^{T}X) + \rho X + \lambda \sum_{k=0}^d A_{k}^{T} (\sum_{h=0}^{d} A_{h} X \Psi_{h}^{T})\Psi_{k} = W \Rho_{\Omega}(Y) $$

*Conjugate gradient for inferring X:*

1. Initialize x as $x_{0}$ and compute the residual $r_{o}:= vec(W \Rho_{\Omega}(Y)) - \mathcal{L}_{X_{0}}$. Let $q_{0} := r_{0}$.
2. Repeat:
   1. Covert $q_{t}$ into matrix $Q_{t}$.
   2. Compute $a_{l}:= r_{l}^{T}r_{l} / q_{l}^{T} \mathcal{L}_{X} (Q_{l}) $ and update:
      1. $ x_{l+1} := x_{l} + \alpha_{l} q_{l} $
      2. $ r_{l+1} := r_{l} - \mathcal{L}_{X} (Q_{l}) $
   3. Compute $\beta_{l} := r_{l+1}^{T}r_{l+1} / r_{l}^{T}r_{l}, and update q_{l+1} := r_{l+1} + \beta_{l}q_{l} $
   4. Update l := l+1

The approximated solution is obtained through conjugate gradient method. As we know, conjugate gradient is an efficient method for solving linear equations.

## Forecasting Mechanism


Until now, we have a NoTMF model for learning unobserved entries from partial observations and performing forecasting. To configure a rolling forecasting task on real-world data, we still need to devise a mechanism in a rolling manner.

At the first rolling time, we can learn a NoTMF model from $Y_{t}$ and gather the parameters {W,X,A}. For the temporal factors we can use vector autoregressive to perform forecasting, estimating x_{t+1}.

Then at the next rolling time, we can fix the variable W and use Y_{t+1} to update the variables {X,A}. It is also possible to perform forecasting in this situation.

Code from: https://github.com/xinychen/tracebase

*Note: the data must be downloaded to recreate experiment*

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('movement-speeds-hourly-new-york-2019-1.csv')
road = data.drop_duplicates(['osm_way_id', 'osm_start_node_id', 'osm_end_node_id'])
road = road.drop(['year', 'month', 'day', 'hour', 'utc_timestamp', 'segment_id', 'start_junction_id', 
                  'end_junction_id', 'speed_mph_mean', 'speed_mph_stddev'], axis = 1)
road.to_csv('road.csv')

**Construct Speed Matrix**

The matrix's row corresponds to one specific road segment, while the column corresponds to one specific hour.

In [ ]:
import numpy as np
import pandas as pd

month = 1
data = pd.read_csv('movement-speeds-hourly-new-york-2019-{}.csv'.format(month))
road = pd.read_csv('road.csv')
tensor = np.zeros((road.shape[0], max(data.day.values), 24))
k = 0
for i in range(road.shape[0]):
    temp = data[(data['osm_way_id'] == road.osm_way_id.iloc[i]) 
                & (data['osm_start_node_id'] == road.osm_start_node_id.iloc[i]) 
                & (data['osm_end_node_id'] == road.osm_end_node_id.iloc[i])]
    for j in range(temp.shape[0]):
        tensor[k, temp.day.iloc[j] - 1, temp.hour.iloc[j]] = temp.speed_mph_mean.iloc[j]
    k += 1
    if (k % 1000) == 0:
        print(k)
mat = tensor.reshape([road.shape[0], max(data.day.values) * 24])
np.savez_compressed('hourly_speed_mat_2019_{}.npz'.format(month), mat)

del data, tensor

**Data Analysis**

If you want to investigate the missing data problem in Uber movement speed data, please prepare the data in the whole year of 2019 by yourself through the above codes. 

**Analyzing Missing Data**

In [ ]:
## Build a speed matrix for the whole year of 2019 in NYC
mat = np.load('hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 13):
    mat = np.append(mat, np.load('hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)

## Calculate missing rates
print('The missing ratte of speed matrix is:')
print(len(np.where(mat == 0)[0]) / (mat.shape[0] * mat.shape[1]))

N, T = mat.shape
sample_rate = np.zeros(T)
for t in range(T):
    pos = np.where(mat[:, t] == 0)
    sample_rate[t] = len(pos[0]) / N
sample_rate = sample_rate[: 52 * 7 * 24].reshape([52, 24 * 7])
whole_rate = np.mean(sample_rate, axis = 0)

**Draw Missing Rates**

In [ ]:
rate = len(np.where(mat == 0)[0]) / (mat.shape[0] * mat.shape[1])
print(rate)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 12
fig = plt.figure(figsize = (8, 2))
ax = fig.add_subplot(1, 1, 1)
plt.plot(whole_rate, color = 'red', linewidth = 1.8)
upper = whole_rate + np.std(sample_rate, axis = 0)
lower = whole_rate - np.std(sample_rate, axis = 0)
x_bound = np.append(np.append(np.append(np.array([0, 0]), np.arange(0, 7 * 24)), 
                              np.array([7 * 24 - 1, 7 * 24 - 1])), np.arange(7 * 24 - 1, -1, -1))
y_bound = np.append(np.append(np.append(np.array([upper[0], lower[0]]), lower), 
                              np.array([lower[-1], upper[-1]])), np.flip(upper))
plt.fill(x_bound, y_bound, color = 'red', alpha = 0.2)
plt.axhline(y = rate, color = 'gray', alpha = 0.5, linestyle='dashed')
plt.xticks(np.arange(0, 24 * 7 + 1, 1 * 24))
plt.xlabel('Time (hour)')
plt.ylabel('Missing rate')
plt.grid(axis = 'both', linestyle='dashed', linewidth = 0.1, color = 'gray')
ax.tick_params(direction = "in")
ax.set_xlim([-1, 7 * 24])
# ax.set_ylim([0.6, 1])
plt.show()
# fig.savefig("NYC_missing_rate_stat.pdf", bbox_inches = "tight")

**Analyze Observation Rate of Road Segments**

In [ ]:
import numpy as np

mat = np.load('hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 13):
    mat = np.append(mat, np.load('hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)
ratio = np.sum(mat > 0, axis = 1) / (365 * 24)

In [ ]:
for threshold in 0.1 * np.arange(1, 10):
    print('Observation rate > {0:.2f}'.format(threshold))
    print(np.sum(ratio > threshold))
    print(np.sum(ratio > threshold) / ratio.shape[0])
    print()

### Start Model Forecasting

**Define evaluation functions**

In [ ]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [ ]:
# Generate temporal operators

def generate_Psi(T, d, season):
    Psi = []
    for k in range(0, d + 1):
        if k == 0:
            Psi.append(np.append(np.zeros((T - d - season, d)), 
                                 np.append(-1 * np.eye(T - d - season), np.zeros((T - d - season, season)), axis = 1) 
                                 + np.append(np.zeros((T - d - season, season)), np.eye(T - d - season), axis = 1), axis = 1))
        else:
            Psi.append(np.append(np.append(np.zeros((T - d - season, d - k)), 
                                           np.append(-1 * np.eye(T - d - season), np.zeros((T - d - season, season)), axis = 1)
                                           + np.append(np.zeros((T - d - season, season)), np.eye(T - d - season), axis = 1), axis = 1), 
                                 np.zeros((T - d - season, k)), axis = 1))
    return Psi

**Define conjugate gradient for factor matrix:**

In [ ]:
def update_cg(var, r, q, Aq, rold):
    alpha = rold / np.inner(q, Aq)
    var = var + alpha * q
    r = r - alpha * Aq
    rnew = np.inner(r, r)
    q = r + (rnew / rold) * q
    return var, r, q, rnew

def ell_w(ind, W, X, rho):
    return X @ ((W.T @ X) * ind).T + rho * W

def conj_grad_w(sparse_mat, ind, W, X, rho, maxiter = 5):
    rank, dim1 = W.shape
    w = np.reshape(W, -1, order = 'F')
    r = np.reshape(X @ sparse_mat.T - ell_w(ind, W, X, rho), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim1), order = 'F')
        Aq = np.reshape(ell_w(ind, Q, X, rho), -1, order = 'F')
        w, r, q, rold = update_cg(w, r, q, Aq, rold)
    return np.reshape(w, (rank, dim1), order = 'F')

def ell_x(ind, W, X, A, Psi, d, lambda0, rho):
    rank, dim2 = X.shape
    temp = np.zeros((d * rank, Psi[0].shape[0]))
    for k in range(1, d + 1):
        temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
    temp1 = X @ Psi[0].T - A @ temp
    temp2 = np.zeros((rank, dim2))
    for k in range(d):
        temp2 += A[:, k * rank : (k + 1) * rank].T @ temp1 @ Psi[k + 1]
    return W @ ((W.T @ X) * ind) + rho * X + lambda0 * (temp1 @ Psi[0] - temp2)

def conj_grad_x(sparse_mat, ind, W, X, A, Psi, d, lambda0, rho, maxiter = 5):
    rank, dim2 = X.shape
    x = np.reshape(X, -1, order = 'F')
    r = np.reshape(W @ sparse_mat - ell_x(ind, W, X, A, Psi, d, lambda0, rho), -1, order = 'F')
    q = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Q = np.reshape(q, (rank, dim2), order = 'F')
        Aq = np.reshape(ell_x(ind, W, Q, A, Psi, d, lambda0, rho), -1, order = 'F')
        x, r, q, rold = update_cg(x, r, q, Aq, rold)
    return np.reshape(x, (rank, dim2), order = 'F')

**Define NonStationary Temporal Matrix Factorization**

In [ ]:
def notmf(dense_mat, sparse_mat, rank, d, lambda0, rho, season, maxiter):
    dim1, dim2 = sparse_mat.shape
    W = 0.01 * np.random.randn(rank, dim1)
    X = 0.01 * np.random.randn(rank, dim2)
    A = 0.01 * np.random.randn(rank, d * rank)
    if np.isnan(sparse_mat).any() == False:
        ind = sparse_mat != 0
        pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    elif np.isnan(sparse_mat).any() == True:
        pos_test = np.where((dense_mat != 0) & (np.isnan(sparse_mat)))
        ind = ~np.isnan(sparse_mat)
        sparse_mat[np.isnan(sparse_mat)] = 0
    dense_test = dense_mat[pos_test]
    del dense_mat
    Psi = generate_Psi(dim2, d, season)
    show_iter = 100
    temp = np.zeros((d * rank, dim2 - d - season))
    for it in range(maxiter):
        W = conj_grad_w(sparse_mat, ind, W, X, rho)
        X = conj_grad_x(sparse_mat, ind, W, X, A, Psi, d, lambda0, rho)
        for k in range(1, d + 1):
            temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
        A = X @ Psi[0].T @ np.linalg.pinv(temp)
        mat_hat = W.T @ X
        if (it + 1) % show_iter == 0:
            temp_hat = mat_hat[pos_test]
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            print()
    return mat_hat, W, X, A

def notmf_dic(obs, W, X, A, d, lambda0, rho, season):
    dim1, dim2 = obs.shape
    rank = X.shape[0]
    if np.isnan(obs).any() == False:
        ind = obs != 0
    elif np.isnan(obs).any() == True:
        ind = ~np.isnan(obs)
        obs[np.isnan(obs)] = 0
    Psi = generate_Psi(dim2, d, season)
    X = conj_grad_x(obs, ind, W, X, A, Psi, d, lambda0, rho)
    temp = np.zeros((d * rank, dim2 - d - season))
    for k in range(1, d + 1):
        temp[(k - 1) * rank : k * rank, :] = X @ Psi[k].T
    A = X @ Psi[0].T @ np.linalg.pinv(temp)
    return X, A

**Define the function of VAR forecasting**

In [ ]:
def var4cast(X, A, d, delta, season):
    dim1, dim2 = X.shape
    X_hat = np.append(X[:, season:] - X[:, : -season], np.zeros((dim1, delta)), axis = 1)
    for t in range(delta):
        X_hat[:, dim2 - season + t] = A @ X_hat[:, dim2 - season + t - np.arange(1, d + 1)].T.reshape(dim1 * d)
    X = np.append(X, np.zeros((dim1, delta)), axis = 1)
    for t in range(delta):
        X[:, dim2 + t] = X[:, dim2 - season + t] + X_hat[:, dim2 - season + t]
    return X

**Define the function for rolling forecasting**

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

def rolling4cast(dense_mat, sparse_mat, pred_step, delta, rank, d, lambda0, rho, season, maxiter):
    dim1, T = sparse_mat.shape
    start_time = T - pred_step
    max_count = int(np.ceil(pred_step / delta))
    mat_hat = np.zeros((dim1, max_count * delta))
    f = IntProgress(min = 0, max = max_count) # instantiate the bar
    display(f) # display the bar
    for t in range(max_count):
        if t == 0:
            _, W, X, A = notmf(dense_mat[:, : start_time], sparse_mat[:, : start_time], 
                               rank, d, lambda0, rho, season, maxiter)
        else:
            X, A = notmf_dic(sparse_mat[:, : start_time + t * delta], W, X_new, A, d, lambda0, rho, season)
        X_new = var4cast(X, A, d, delta, season)
        mat_hat[:, t * delta : (t + 1) * delta] = W.T @ X_new[:, - delta :]
        f.value = t
    small_dense_mat = dense_mat[:, start_time : T]
    pos = np.where((small_dense_mat != 0) & (np.invert(np.isnan(small_dense_mat))))
    mape = compute_mape(small_dense_mat[pos], mat_hat[pos])
    rmse = compute_rmse(small_dense_mat[pos], mat_hat[pos])
    print('Prediction MAPE: {:.6}'.format(mape))
    print('Prediction RMSE: {:.6}'.format(rmse))
    print()
    return mat_hat, W, X, A

**Test on the dataset**

In [ ]:
import numpy as np

dense_mat = np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_1.npz')['arr_0']
for month in range(2, 4):
    dense_mat = np.append(dense_mat, np.load('../datasets/NYC-movement-data-set/hourly_speed_mat_2019_{}.npz'.format(month))['arr_0'], axis = 1)

import time
for rank in [10]: # rank of matrix factorization
    for delta in [1, 2, 3, 6]: # forecasting time horizon
        for d in [6]: # order of VAR
            start = time.time()
            pred_step = 7 * 24
            lambda0 = 1
            rho =  5
            season = 7 * 24
            maxiter = 50
            mat_hat, W, X, A = rolling4cast(dense_mat[:, : 24 * 7 * 10], dense_mat[:, : 24 * 7 * 10], 
                                            pred_step, delta, rank, d, lambda0, rho, season, maxiter)
            print('delta = {}'.format(delta))
            print('rank R = {}'.format(rank))
            print('Order d = {}'.format(d))
            end = time.time()
            print('Running time: %d seconds'%(end - start))